In [3]:
!pip install backtrader
!pip install shioaji
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.dates as mdates
import datetime
from datetime import timedelta
from matplotlib.dates import date2num
import backtrader as bt
from google.colab import userdata

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

#==== for Colab
shioaji_secret=userdata.get('SHIOAJI_SECRETKEY')
shioaji_apikey=userdata.get('SHIOAJI_APIKEY')

# 登入你的key
accounts = api.login(shioaji_apikey, shioaji_secret)




Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [43]:
# 取得一週的成值排行
firday= datetime.date(2024,8,2)
monday = firday - timedelta(days=4)
def get_weekly_amount_rank_list(firday):
  amount_rank_dfs=[]
  for i in range(5):
    day = firday - timedelta(days=i)
    # 成值排行
    scanners = api.scanners(
          scanner_type=sj.constant.ScannerType.AmountRank,
          count=100,
          date=str(day)
      )
    amount_rank_df = pd.DataFrame(s.__dict__ for s in scanners)
    if amount_rank_df.empty:
      continue
    else:
      amount_rank_df.ts = pd.to_datetime(amount_rank_df.ts)
    new_amount_rank_df=amount_rank_df[["code","name","rank_value"]]
    amount_rank_dfs.append(new_amount_rank_df)

  amount_rank_weekly=pd.concat(amount_rank_dfs, join='inner')
  weekly_amount_rank_df = amount_rank_weekly.groupby('code').agg({
      'name': 'first',        # 保留每個 group 的第一個 name
      'rank_value': 'sum'     # 對 rank_value 做加總
  }).reset_index()

  weekly_amount_rank_df.sort_values(by='rank_value', ascending=False, inplace=True)
  # 週成交金額>10億
  weekly_amount_rank = weekly_amount_rank_df[weekly_amount_rank_df['rank_value'] > 1.e+9]
  weekly_amount_rank_code = weekly_amount_rank['code'].tolist()
  return weekly_amount_rank_code

weekly_amount_rank_list=get_weekly_amount_rank_list(firday)
print(weekly_amount_rank_list)



def get_weekly_data(stock_code):
  # 取得日 K 資料
  kbars = api.kbars(
      contract=api.Contracts.Stocks[stock_code],
      start=str(monday),
      end=str(firday),
  )

  # 將資料轉換為 DataFrame
  df = pd.DataFrame({**kbars})
  df['ts'] = pd.to_datetime(df['ts'])

  # 按週分組，並計算每週的開盤、最高、最低、收盤和成交量
  df.set_index('ts', inplace=True)

  # 定義週 K 的聚合方式
  weekly_kbars = df.resample('W').agg({
      'Open': 'first',       # 每週第一天的開盤價
      'High': 'max',         # 每週最高價
      'Low': 'min',          # 每週最低價
      'Close': 'last',       # 每週最後一天的收盤價
      'Volume': 'sum'        # 每週總成交量
  }).dropna()  # 移除有空值的週
  weekly_kbars['close_open_diff_percentage'] = ((weekly_kbars['Close'] - weekly_kbars['Open']) / weekly_kbars['Open'] * 100).round(2)
  # 加入 stock_code 欄位，並設置為第一欄
  weekly_kbars.insert(0, 'stock_code', stock_code)
  return weekly_kbars

def get_weekly_good_raise_stock_df(weekly_amount_rank_code_list):
  final_good_weekly_stock = []
  for stock in weekly_amount_rank_code_list:
    weekly_kbars = get_weekly_data(stock)
    if weekly_kbars['close_open_diff_percentage'].values[0] >= 15:
      final_good_weekly_stock.append(stock)
    else:
      continue
  dfs = []
  for good_stock in final_good_weekly_stock:
    contract=api.Contracts.Stocks[good_stock]
    # 將合約物件的屬性轉換為字典
    contract_dict = {
        'code': contract.code,
        'name': contract.name
    }
    # 將字典轉換為 DataFrame
    df_contract = pd.DataFrame([contract_dict])
    dfs.append(df_contract)

  # 使用 concat 將所有 DataFrame 合併成一個大的 DataFrame
  final_df = pd.concat(dfs, ignore_index=True)
  return final_df




# print(get_weekly_good_raise_stock_df(weekly_amount_rank_list))

['2330', '2317', '2454', '2382', '3017', '2308', '3037', '6669', '3661', '5443', '8054', '3443', '2303', '3374', '1519', '8028', '3293', '3324', '2603', '3035', '3663', '3163', '3008', '2365', '2359', '3711', '1513', '3189', '6869', '2327', '2383', '2301', '6446', '2486', '0050', '3231', '2376', '2449', '2609', '3376', '2363', '2357', '2891', '3481', '8064', '3034', '3450', '2524', '3583', '2337', '6462', '2348', '8069', '6640', '1503', '6239', '2368', '2881', '5508', '3665', '2615', '6805', '6643', '2356', '9945', '6187', '6223', '2882', '9906', '3047', '2412', '3533', '1216', '2379', '3580', '6806', '0056', '3105', '3013', '5347', '9958', '4763', '5274', '3131', '2464', '2349', '5269', '2313', '8996', '6125', '6274', '4958', '3653', '8299', '4513', '1514', '2888', '6415', '2059', '2345', '1590', '6472', '2542', '3535', '2467', '4968', '2204', '2362', '2884', '3211', '1560', '3491', '8046', '6488', '2886', '8374', '1609', '1316', '3715', '2360', '4562', '3406', '6531', '2408', '4967',

In [44]:
firday= datetime.date(2024,2,2)
monday = firday - timedelta(days=4)
print(get_weekly_good_raise_stock_df(weekly_amount_rank_list))

    code name
0   3017   奇鋐
1   8054   安國
2   1519   華城
3   3324   雙鴻
4   2376   技嘉
5   3376  新日興
6   6462   神盾
7   6805  富世達
8   6643  M31
9   8996   高力
10  6274   台燿


In [ ]:


# 取得日K資料
def get_daily_kbar(stock_code, start_date, end_date):
    kbars = api.kbars(
        contract=api.Contracts.Stocks[stock_code],
        start=start_date,
        end=end_date,
    )
    # 將 'ts' 列重命名為 'datetime'
    df = pd.DataFrame({**kbars}).rename(columns={'ts': 'datetime', 'Open': 'open', 'Close': 'close', 'High': 'high', 'Low': 'low', 'Volume': 'volume'})

    # 將 'datetime' 轉換為日期時間格式
    df['datetime'] = pd.to_datetime(df['datetime'])

    # 依照日期分組，並計算每組的 open, close, high, low, volume
    daily_kbar = df.resample('D', on='datetime').agg({
        'open': 'first',
        'close': 'last',
        'high': 'max',
        'low': 'min',
        'volume': 'sum'  # 計算每日成交量的總和
    }).dropna()  # 移除空白的天數

    # 計算當日漲跌幅
    daily_kbar['previous_close'] = daily_kbar['close'].shift(1)

    # 計算今天的漲跌幅 (今天的 close - 昨天的 close) / 昨天的 close * 100
    daily_kbar['change_percentage'] = ((daily_kbar['close'] - daily_kbar['previous_close']) / daily_kbar['previous_close'] * 100).round(2)

    # 計算當日震幅
    daily_kbar['amplitude'] = ((daily_kbar['high'] - daily_kbar['low']) / daily_kbar['high'] * 100).round(2)

    # 移除前一天沒有數據的行
    daily_kbar = daily_kbar.dropna(subset=['previous_close'])

    print(daily_kbar)

    # 將結果輸出到 Excel 檔案
    # daily_kbar.to_excel(f'{stock_code}-daily_kbar.xlsx', index=True)  # `index=True` 會將日期作為列索引輸出

    return daily_kbar


In [ ]:

start_day ="2021-01-01"
end_day = "2023-12-31"

# goodStockList2020=[4743,6443,6531,4128,3545 ,6547,2609 ,8046 ,3218 ,6492 ,2368 ,2603 ,4961 ,2303 ,2615 ,3661 ,5269 ,4142 ,6415, 3374]
# for(stock_code)in goodStockList2020:
#     stock_code = str(stock_code)
#     get_daily_kbar(stock_code,start_day,end_day)

# get_daily_kbar(f"{stock_code}",start_day,end_day)

df_2609 = get_daily_kbar("1519",start_day,end_day)

              open   close    high     low  volume  previous_close  \
datetime                                                             
2021-01-05   48.95   48.75   49.40   48.70    1247           49.00   
2021-01-06   48.70   46.85   48.95   46.55    3972           48.75   
2021-01-07   46.95   47.80   49.45   46.95    3628           46.85   
2021-01-08   48.25   48.50   48.50   46.80    2400           47.80   
2021-01-11   48.70   48.30   49.30   48.15    2095           48.50   
...            ...     ...     ...     ...     ...             ...   
2023-12-25  326.00  333.50  341.00  326.00   12052          321.00   
2023-12-26  338.50  315.00  340.50  314.00   15301          333.50   
2023-12-27  316.50  316.50  323.50  316.50    8549          315.00   
2023-12-28  325.00  348.00  348.00  325.00   18627          316.50   
2023-12-29  358.00  327.00  360.50  317.00   34502          348.00   

            change_percentage  amplitude  
datetime                                  
202

In [ ]:
# sma cross strategy
class DailyMAStrategy(bt.Strategy):
  # 設定參數
  params = (('maperiods',[5,10,20,60]),)

  def __init__(self):
    # 定義移動平均線（5MA, 10MA, 20MA, 60MA）
    self.sma5 = bt.ind.SMA(self.data.close, period=self.params.maperiods[0])
    self.sma10 = bt.ind.SMA(self.data.close, period=self.params.maperiods[1])
    self.sma20 = bt.ind.SMA(self.data.close, period=self.params.maperiods[2])
    self.sma60 = bt.ind.SMA(self.data.close, period=self.params.maperiods[3])


  def next(self):
    # 尾盤的時間點 (假設是當日最後一個時間點)
    if len(self.data) < 2:  # 保證有足夠的前一日數據
      return

    # 買入條件
    if not self.position:
      if (self.data.close[0] > self.sma5[0] and
          self.data.close[0] > self.sma10[0] and
          self.data.close[0] > self.sma20[0] and
          self.data.close[0] > self.sma60[0] and
          self.data.volume[0] > 3* self.data.volume[-1]):

          self.buy(size=3000)
          print(f"Buy at {self.data.close[0]} on {self.data.datetime.date(0)}")

    elif self.position:
      if self.data.close[0] < self.sma60[0]:
        print(f"Sell at {self.data.close[0]} on {self.data.datetime.date(0)}")
        self.sell(size=3000)






In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(DailyMAStrategy)
cerebro.broker.setcash(10**6)
data = bt.feeds.PandasData(dataname=df_2609)
cerebro.adddata(data)

# 券商的手續費
cerebro.broker.setcommission(commission=0.1425/100)



# 如果跑到最後，還有股票沒有賣出的話，會以當天收盤價的價格來加總
print("初始資金: %.2f" % cerebro.broker.getvalue())

# 執行策略
cerebro.run()

print("結束資金：%.2f" % cerebro.broker.getvalue())
# # 可視化結果
cerebro.plot()

初始資金: 1000000.00
Buy at 44.3 on 2021-07-13
Sell at 43.85 on 2021-07-14
Buy at 36.95 on 2021-12-15
Sell at 36.15 on 2022-01-07
Buy at 39.6 on 2022-01-11
Sell at 36.45 on 2022-02-24
Buy at 38.7 on 2022-03-03
Sell at 37.0 on 2022-03-08
Buy at 42.05 on 2022-03-14
Sell at 38.9 on 2022-04-25
Buy at 37.0 on 2022-07-14
Sell at 39.0 on 2022-10-20
Buy at 40.5 on 2022-11-21
Sell at 39.95 on 2022-11-23
Buy at 42.7 on 2022-12-02
Sell at 251.0 on 2023-10-27
Buy at 275.0 on 2023-11-24
結束資金：1748341.27


[[<Figure size 640x480 with 4 Axes>]]

In [41]:
print(api.usage())

connections=3 bytes=358425150 limit_bytes=2147483648 remaining_bytes=1789058498
